In [1]:
import cv2
import numpy as np
import os

In [2]:
def urutkan_titik(pts):
    """Urutkan titik jadi [Top-Left, Top-Right, Bottom-Right, Bottom-Left]"""
    rect = np.zeros((4, 2), dtype="float32")
    s = pts.sum(axis=1)
    rect[0] = pts[np.argmin(s)]   # top-left
    rect[2] = pts[np.argmax(s)]   # bottom-right
    diff = np.diff(pts, axis=1)
    rect[1] = pts[np.argmin(diff)]  # top-right
    rect[3] = pts[np.argmax(diff)]  # bottom-left
    return rect


In [3]:
def four_point_transform(image, pts):
    """Lakukan transformasi perspektif"""
    rect = urutkan_titik(pts)
    (tl, tr, br, bl) = rect
    widthA = np.linalg.norm(br - bl)
    widthB = np.linalg.norm(tr - tl)
    maxWidth = max(int(widthA), int(widthB))
    heightA = np.linalg.norm(tr - br)
    heightB = np.linalg.norm(tl - bl)
    maxHeight = max(int(heightA), int(heightB))
    dst = np.array([
        [0, 0],
        [maxWidth - 1, 0],
        [maxWidth - 1, maxHeight - 1],
        [0, maxHeight - 1]], dtype="float32")
    M = cv2.getPerspectiveTransform(rect, dst)
    warp = cv2.warpPerspective(image, M, (maxWidth, maxHeight))
    return warp


In [4]:
def proses_gambar(path_input, path_output):
    image = cv2.imread(path_input)
    if image is None:
        print(f"Gagal membaca gambar: {path_input}")
        return

    nama_file = os.path.splitext(os.path.basename(path_input))[0]

    orig = image.copy()
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    gray = cv2.GaussianBlur(gray, (5, 5), 0)

    # Deteksi tepi
    edged = cv2.Canny(gray, 75, 200)

    # Temukan kontur
    contours, _ = cv2.findContours(edged.copy(), cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
    contours = sorted(contours, key=cv2.contourArea, reverse=True)[:5]

    doc_cnt = None
    for c in contours:
        peri = cv2.arcLength(c, True)
        approx = cv2.approxPolyDP(c, 0.02 * peri, True)
        if len(approx) == 4:
            doc_cnt = approx.reshape(4, 2)
            break

    if doc_cnt is None:
        print(f"❌ Dokumen tidak ditemukan pada {nama_file}")
        return

    rect = urutkan_titik(doc_cnt)
    labels = ["TL", "TR", "BR", "BL"]

    # Gambar titik koordinat
    for i, (x, y) in enumerate(rect):
        x, y = int(x), int(y)
        cv2.circle(image, (x, y), 8, (0, 0, 255), -1)
        teks = f"{labels[i]} ({x},{y})"
        cv2.putText(image, teks, (x + 10, y - 10), cv2.FONT_HERSHEY_SIMPLEX,
                    0.6, (255, 255, 255), 2, cv2.LINE_AA)
        cv2.putText(image, teks, (x + 10, y - 10), cv2.FONT_HERSHEY_SIMPLEX,
                    0.6, (0, 0, 0), 1, cv2.LINE_AA)

    # Gambar garis antar titik
    pts = rect.astype(int)
    cv2.polylines(image, [pts], isClosed=True, color=(0, 255, 0), thickness=2)

    # Simpan hasil anotasi
    output_annotated = os.path.join(path_output, f"{nama_file}_annotated.jpg")
    cv2.imwrite(output_annotated, image)
    print(f"✅ {nama_file}: Gambar dengan koordinat disimpan di {output_annotated}")

    # Tampilkan hasil
    cv2.imshow("Koordinat Dokumen", image)
    cv2.waitKey(500)  # tampil 0.5 detik per gambar

    # Transformasi perspektif (scan)
    warped = four_point_transform(orig, rect)
    warped_gray = cv2.cvtColor(warped, cv2.COLOR_BGR2GRAY)
    warped_clean = cv2.adaptiveThreshold(
        warped_gray, 255,
        cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 15, 10
    )

    output_scan = os.path.join(path_output, f"{nama_file}_scan.jpg")
    cv2.imwrite(output_scan, warped_clean)
    print(f"🧾 {nama_file}: Hasil scan bersih disimpan di {output_scan}\n")

In [5]:
def main():
    folder_input = "data_mentah"
    folder_output = "data_output"

    if not os.path.exists(folder_output):
        os.makedirs(folder_output)

    file_list = [f for f in os.listdir(folder_input)
                 if f.lower().endswith(('.jpg', '.jpeg', '.png', '.bmp', '.tiff'))]

    if not file_list:
        print("❌ Tidak ada gambar di folder data_mentah/")
        return

    for file_name in file_list:
        path_in = os.path.join(folder_input, file_name)
        proses_gambar(path_in, folder_output)

    cv2.destroyAllWindows()
    print("🚀 Semua gambar selesai diproses!")

if __name__ == "__main__":
    main()

✅ data_a1: Gambar dengan koordinat disimpan di data_output\data_a1_annotated.jpg
🧾 data_a1: Hasil scan bersih disimpan di data_output\data_a1_scan.jpg

✅ data_a10: Gambar dengan koordinat disimpan di data_output\data_a10_annotated.jpg
🧾 data_a10: Hasil scan bersih disimpan di data_output\data_a10_scan.jpg

❌ Dokumen tidak ditemukan pada data_a100
✅ data_a101: Gambar dengan koordinat disimpan di data_output\data_a101_annotated.jpg
🧾 data_a101: Hasil scan bersih disimpan di data_output\data_a101_scan.jpg

❌ Dokumen tidak ditemukan pada data_a102
❌ Dokumen tidak ditemukan pada data_a103
✅ data_a104: Gambar dengan koordinat disimpan di data_output\data_a104_annotated.jpg
🧾 data_a104: Hasil scan bersih disimpan di data_output\data_a104_scan.jpg

❌ Dokumen tidak ditemukan pada data_a105
❌ Dokumen tidak ditemukan pada data_a107
❌ Dokumen tidak ditemukan pada data_a108
❌ Dokumen tidak ditemukan pada data_a109
✅ data_a11: Gambar dengan koordinat disimpan di data_output\data_a11_annotated.jpg
🧾 